In [2]:
# Загрузка и обработка данных
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords

# Визуализация данных
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

pd.options.display.max_columns = None
warnings.filterwarnings("ignore")
nltk.download('popular')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/artemka/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/artemka/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/artemka/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/artemka/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/artemka/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/artemka/nltk_data...
[nltk_data]    |   Package movie_reviews is already

True

In [3]:
dates_columns = ['date', 'createdDate', 'changedDate']
data = pd.read_csv('../data/prepared/outcomes.csv', parse_dates=dates_columns)

In [4]:
data.tail()

,date,categoryName,payee,comment,createdDate,changedDate,outcomeAccountName,outcome,outcomeCurrencyShortTitle
1187,2024-03-20,Проезд,Самолет,Билеты домой и из дома,2024-03-20 12:39:37,2024-03-20 09:39:52,Тинькофф,21020.0,RUB
1188,2024-03-20,Кафе и рестораны,Столовая ОЦРВ,NaN,2024-03-20 12:41:05,2024-03-20 09:41:14,Тинькофф,460.0,RUB
1189,2024-03-20,Кафе и рестораны,Ресторан Вега,NaN,2024-03-21 12:53:07,2024-03-21 09:53:20,Тинькофф,265.0,RUB
1190,2024-03-20,Проезд,Яндекс такси,NaN,2024-03-21 12:53:17,2024-03-21 09:53:37,Тинькофф,212.0,RUB
1191,2024-03-21,Кафе и рестораны,Столовая ОЦРВ,NaN,2024-03-21 12:53:35,2024-03-21 09:53:46,Тинькофф,460.0,RUB


## Посмотрим на выбросы в данных

In [5]:
px.box(data, y='outcome')

Мы видим 1 очень выделяющийся выброс, связанный с покупкой курса по аналитике за 80 тыс. рублей. Я думаю, что его стоит удалить, чтобы он не смещал дальнейшие графики

In [6]:
data = data[data['outcome'] < 80000]

## Рассчитаем базовые статистики

In [7]:
mean = data['outcome'].mean()
median = data['outcome'].median()
percentile_20 = data['outcome'].quantile(q=0.2)
percentile_40 = data['outcome'].quantile(q=0.4)
percentile_60 = data['outcome'].quantile(q=0.6)
percentile_80 = data['outcome'].quantile(q=0.8)
maximun = data['outcome'].max()
minimun = data['outcome'].min()
standard_deviation = data['outcome'].std()

result = f"""
<< Базовая статистика >>

+ Средняя сумма трат: {mean} руб

+ Медианная сумма трат: {median} руб
+ 20% моих трат меньше, чем {percentile_20} рублей
+ 40% моих трат меньше, чем {percentile_40} рублей
+ 60% моих трат меньше, чем {percentile_60} рублей
+ 80% моих трат меньше, чем {percentile_80} рублей

+ Максималная трата: {maximun} руб
+ Минимальная трата: {minimun} руб

* Стандартное отклонение: {standard_deviation} руб
"""
print(result)


<< Базовая статистика >>

+ Средняя сумма трат: 455.6634844668346 руб

+ Медианная сумма трат: 259.98 руб
+ 20% моих трат меньше, чем 110.5 рублей
+ 40% моих трат меньше, чем 200.0 рублей
+ 60% моих трат меньше, чем 323.0 рублей
+ 80% моих трат меньше, чем 533.0 рублей

+ Максималная трата: 21020.0 руб
+ Минимальная трата: 10.0 руб

* Стандартное отклонение: 876.8872978496819 руб



Теперь мне интересно посмотреть на эти басовые статистики не в целом, а по дням

In [8]:
data_by_days = data.groupby('date', as_index=False) \
                  .agg({'outcome' : 'sum'})
all_dates = pd.date_range(start=data_by_days['date'].min(), end=data_by_days['date'].max(), freq='D').to_frame(index=False, name='date')

data_by_days = pd.merge(all_dates, data_by_days, on='date', how='left').fillna({'outcome': 0})

mean = data_by_days['outcome'].mean()
median = data_by_days['outcome'].median()
percentile_20 = data_by_days['outcome'].quantile(q=0.2)
percentile_40 = data_by_days['outcome'].quantile(q=0.4)
percentile_60 = data_by_days['outcome'].quantile(q=0.6)
percentile_80 = data_by_days['outcome'].quantile(q=0.8)
maximun = data_by_days['outcome'].max()
minimun = data_by_days['outcome'].min()
standard_deviation = data_by_days['outcome'].std()

result = f"""
<< Базовая статистика за день>>

+ Средняя сумма трат (в день): {mean} руб

+ Медианная сумма (в день) трат: {median} руб
+ 20% моих трат (в день) меньше, чем {percentile_20} рублей
+ 40% моих трат (в день) меньше, чем {percentile_40} рублей
+ 60% моих трат (в день) меньше, чем {percentile_60} рублей
+ 80% моих трат (в день) меньше, чем {percentile_80} рублей

+ Максималная трата за день: {maximun} руб
+ Минимальная трата за день: {minimun} руб

* Стандартное отклонение: {standard_deviation} руб
"""
print(result)


<< Базовая статистика за день>>

+ Средняя сумма трат (в день): 1462.7903234501346 руб

+ Медианная сумма (в день) трат: 927.51 руб
+ 20% моих трат (в день) меньше, чем 363.7 рублей
+ 40% моих трат (в день) меньше, чем 778.97 рублей
+ 60% моих трат (в день) меньше, чем 1194.98 рублей
+ 80% моих трат (в день) меньше, чем 2009.4 рублей

+ Максималная трата за день: 22039.0 руб
+ Минимальная трата за день: 0.0 руб

* Стандартное отклонение: 2030.589365446051 руб



Тут все уже не так радужно. Медианно, я трачу около 1000 рублей в день, что довольно много. Всего 20% дней я тратил меньше, чем 364 рубля.

## Посмотрим на динамику трат с течением времени

In [9]:
px.line(data_by_days, x='date', y='outcome',
        title='Динамика расходов с течением времени')

В среднем, сумма трат довольно стабильная, но есть редкие выбросы, связанные с крупными покупками

#### Рассмотрим тренд моих трат

In [10]:
plot_data = data.groupby('date', as_index=False) \
                .agg({'outcome' : 'sum'}) \
                .sort_values('date')

plot_data['rolling'] = plot_data['outcome'].rolling(10, center=True).mean()

# Используем px.scatter для добавления линии тренда к исходным данным дохода
fig = px.scatter(plot_data, x='date', y='outcome', trendline="ols",
                 title='Динамика моих расходов с течением времени (Скользящее среднее)')

# Добавляем линию скользящего среднего отдельно
fig.add_scatter(x=plot_data['date'], y=plot_data['rolling'], mode='lines', name='Скользящее среднее')

fig.show()

Итого, мы можем увидеть, что мои все-таки возрасли с марта 2023 года. 

###  Изучим комментарии для трат

In [11]:
print(data[data['comment'].isna() == False]['comment'].shape)
data[data['comment'].isna() == False].head()

(496,)


,date,categoryName,payee,comment,createdDate,changedDate,outcomeAccountName,outcome,outcomeCurrencyShortTitle
1,2023-03-17,Продукты / Сладости,NaN,Купил конфеты Нильс,2023-03-17 20:38:41,2023-03-17 17:38:49,Тинькофф,62.6,RUB
2,2023-03-18,Продукты,NaN,Купил сухой завтрак (50 рублей оплатил баллами),2023-03-18 16:07:54,2023-03-18 13:08:01,Тинькофф,167.0,RUB
3,2023-03-18,Продукты / Сладости,NaN,Купил пудинг и конфеты,2023-03-18 16:08:50,2023-03-18 13:08:55,Тинькофф,214.0,RUB
4,2023-03-19,Кафе и рестораны,ДоДо пицца,Заказал пиццу на ужин (Скинулись с друзьями),2023-03-19 17:32:21,2023-03-19 14:32:28,Тинькофф,342.0,RUB
6,2023-03-20,Кафе и рестораны / Кофейни,NaN,Взял кофе в автомате,2023-03-20 10:33:25,2023-03-20 07:33:33,Тинькофф,100.0,RUB


Суммарно, я оставил комментарий к 496 тратам

In [12]:
symbols_to_remove = ['.', ',', '(', ')', '+', '"']
all_comments = ' '.join(data[data['comment'].isna() == False]['comment'].to_list()).lower().replace('  ', ' ')
for symbol in symbols_to_remove:
    all_comments = all_comments.replace(symbol, '')

all_comments = [word for word in all_comments.split() if word not in stopwords.words('russian')]
Counter(all_comments).most_common(20)

[('обед', 22),
 ('вода', 16),
 ('завтрак', 14),
 ('ужин', 14),
 ('оплатил', 13),
 ('нике', 13),
 ('кофе', 12),
 ('подписку', 12),
 ('дзен', 12),
 ('мани', 12),
 ('ники', 12),
 ('чай', 11),
 ('продукты', 11),
 ('яндекс', 10),
 ('плюс', 10),
 ('такси', 10),
 ('подписка', 9),
 ('оплата', 9),
 ('билет', 9),
 ('кино', 9)]

Итого, в топ 4 самых популярных слов среди моих трат вошли слова "Завтрак", "Обед" и "Ужин", что может указывать на то, что я относительно часто кушаю вне дома. Почетное второе место заняло слово "Вода", поскольку без нее тяжело. 

Ещё одно наблюдение - при должном внимании можно заметить, что в топ 20 вошли слова "ники" и "нике". Суммарно их частота составит 25 повторений, а это однозначно первое место. Видимо, наибольшее количество значимых/необычных трат я совершаю в какой-то связи со своей девушкой.

##### **Попробуем выяснить, в каких ситуациях я чаще оставляю комментарии для трат**

In [13]:
data[data['comment'].isna() == False] \
                    .groupby('categoryName', as_index=False) \
                    .agg({'createdDate' : 'count',
                          'outcome' : 'mean'}) \
                    .sort_values('createdDate', ascending=False)

,categoryName,createdDate,outcome
4,Кафе и рестораны,66,584.921212
20,Продукты,61,251.985902
21,Продукты / Сладости,56,266.305179
5,Кафе и рестораны / Кофейни,38,329.594211
2,Забота о себе,35,493.167429
26,Хобби,30,1440.896000
12,Отдых и развлечения,30,756.533000
23,Проезд,28,1326.895714
3,Здоровье и фитнес,25,1174.406000
18,Подарки,25,1718.157200


Итого, чаще всего я оставляю комментарии для оплаты общепитов, а также продуктов или сладостей. Также, довольно часто я оставляю комментарии для трат на отдых и развлечения, а также для подарков. 

Как итог, могу предположить, что наличие комментария к трате является признаком необычной/значимой траты.

In [14]:
print(f"Медианная сумма трат с комментарием: {data[data['comment'].isna() == False]['outcome'].median()} \n"
      f"Медианная сумма трат без комментариев: {data[data['comment'].isna() == True]['outcome'].median()} \n\n"
      f"Средняя сумма трат с комментарием: {data[data['comment'].isna() == False]['outcome'].mean()} \n"
      f"Средняя сумма трат без комментария: {data[data['comment'].isna() == True]['outcome'].mean()} \n")

Медианная сумма трат с комментарием: 340.0 
Медианная сумма трат без комментариев: 230.0 

Средняя сумма трат с комментарием: 674.8532258064515 
Средняя сумма трат без комментария: 299.23454676258996 



Базовые статистики суммы трат (медиана и среднее) подтверждают мою гипотезу о том, что комментарий является меткой того, что трата неординарная.
Пускай медианные значения и довольно схожи, однако среднее занчение у трат без комментариев и с комментариями отличаются более чем в 2 раза. Это значит, что среди трат с комментариями есть яркие выбросы, в то время, как траты без комментариев довольно равномерные

In [15]:
plot_data = data.copy()
plot_data['has_comment'] = plot_data['comment'].isna() == False

px.scatter(plot_data, x='date', y='outcome', color='has_comment')